## Here we demonstrate how to turn the Moral Machines data into the structure of our games.

In [4]:
import pandas as pd
from scipy import stats

### Load in the Moral Machines Stats on Each Country

In [18]:
mm_countries_df = pd.read_csv('../../external_data/moral_machines/CountriesChangePr.csv')
mm_countries_df.columns
mm_countries_df = mm_countries_df.rename(columns={'Unnamed: 0' : 'ISO3_code'})
mm_countries_df
categories = ['[Omission -> Commission]', 
'[Passengers -> Pedestrians]', 
'Law [Illegal -> Legal]', 
'Gender [Male -> Female]', 
'Fitness [Large -> Fit]', 
'Social Status [Low -> High]', 
'Age [Elderly -> Young]', 
'No. Characters [Less -> More]', 
'Species [Pets -> Humans]']
se_str = ": se"
est_str = ": Estimates"

queries = [f'`{cat + se_str}` < .05' for cat in categories]
query_str = ' & '.join(queries)
mm_countries_sig = mm_countries_df.query(query_str)

# # The following is not necessary as we load these datasets after and query based on what is present in the MM data
# countries_w_data = mm_countries_sig['ISO3_code'].apply(lambda x: x in countries_pop_df.index and x in countries_count.index)
# filtered_countries = mm_countries_sig[countries_w_data]['ISO3_code']
# len(filtered_countries)

query_countries = mm_countries_sig['ISO3_code'] # This is a list of the countries which appear in the relevant data set

### Get the counts for each country (requires all Moral Machines data)

In [5]:
if not os.path.isfile('../../external_data/moral_machines/country_totals.csv'):
    chunksize = 10 ** 6
    count = pd.Series()
    with pd.read_csv('../../external_data/moral_machines/SharedResponses.csv',
                     chunksize=chunksize) as reader:
        for chunk in reader:
            # We are double counting here, I think, but that should be ok because we're not using the counts
            # directly but rather as a proportion
            count = count.add(chunk['UserCountry3'].value_counts(), fill_value=0)
    count.name = 'total rows'
    count.to_csv('../../external_data/moral_machines/country_totals.csv')

/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_91894/1253985769.py:5: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_91894/1253985769.py:5: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_91894/1253985769.py:5: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_91894/1253985769.py:5: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_91894/1253985769.py:5: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=

In [25]:
counts_df = pd.read_csv('../../external_data/moral_machines/country_totals.csv')
countries_count = counts_df[counts_df['UserCountry3'].isin(query_countries)]
countries_count = countries_count.set_index('UserCountry3')
countries_count

,total rows
UserCountry3,
ALB,13913.0
ARE,94743.0
ARG,474543.0
AUS,1864754.0
AUT,624532.0
...,...
URY,70383.0
USA,17850148.0
VEN,90873.0


### Calculate the proportion of each country using UN population data

In [19]:
pop_df = pd.read_csv('../../external_data/WPP2022_Demographic_Indicators_Medium.csv')

# To extract a probability for the given countries from their population
countries_pop_df = pop_df[(pop_df['ISO3_code'].isin(query_countries)) &
                          (pop_df['Time'] == 2022)][['ISO3_code', 'TPopulation1Jan']]
countries_pop_df = countries_pop_df.set_index('ISO3_code')
total_pop = countries_pop_df.sum()
# countries_pop_df.divide(total_pop)
countries_pop_df

/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_91894/2915646155.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pop_df = pd.read_csv('../../external_data/WPP2022_Demographic_Indicators_Medium.csv')


,TPopulation1Jan
ISO3_code,
REU,970.131
DZA,44543.592
EGY,110132.806
MAR,37264.469
TUN,12308.697
...,...
VEN,28047.658
CAN,38290.846
USA,337499.479


## Put the Data into our Framework

In [11]:
import itertools
import random
import os, sys
import value_aggregation as pm

In [12]:
filtered_countries = mm_countries_sig['ISO3_code']
country_combos = list(itertools.combinations(filtered_countries, min(5, len(filtered_countries))))

In [20]:
sample_matchups = random.sample(country_combos, min(100, len(country_combos)))

credence_df = countries_pop_df # `countries_pop_df` or `countries_count`
disagreements = 0
count = 0
results = {}

# First just test each category and its negation
for category in categories:
    # Just a placeholder to test all of the country samples right now
    for matchup in sample_matchups:
        outcomes = {f'support' : {}, f'oppose' : {}}
        beliefs = {}
        for country in matchup:
            # then test subsets of the categories
            util = mm_countries_sig[mm_countries_sig['ISO3_code'] == country][category + est_str].item()
            outcomes[f'support'][country] = util
            outcomes[f'oppose'][country] = -util
            # nb: not normalize because it should be handled
            beliefs[country] = credence_df.loc[country].item()
        gameState = pm.VoteGameState(beliefs, outcomes)
        mec_ans = pm.run_mec(gameState, none_if_tie=True)
        nbs_ans = pm.run_nash_bargain(gameState, none_if_tie=True)

        if mec_ans not in results:
            results[mec_ans] = 0
        if nbs_ans not in results:
            results[nbs_ans] = 0

        results[mec_ans] += 1

        if mec_ans != nbs_ans:
            disagreements += 1
            results[nbs_ans] += 1

        count += 1
        
print(f'proportion disagree: {disagreements / count}')
print()
print(results)

proportion disagree: 0.0

{'support': 896, 'oppose': 4}


In [32]:
def test_disagreements(category_combos, sample_matchups, util_df, credence_df):
    disagreements = 0
    count = 0
    results = {}
    example_disagrees = []

    # Just a placeholder to test all of the country samples right now
    for matchup in sample_matchups:
        # The way to reduce redundancy with above code is to pass a list of element one here
        for cat_combo in category_combos:
            #######
            outcomes = {}
            beliefs = {}
            for category in cat_combo:
                outcomes[category] = {}

            for country in matchup:
                # nb: not normalize because it should be handled
                beliefs[country] = credence_df.loc[country].item()

                for category in cat_combo:
                    util = util_df[util_df['ISO3_code'] == country][category + est_str].item()
                    outcomes[category][country] = util

            gameState = pm.VoteGameState(beliefs, outcomes)
            mec_ans = pm.run_mec(gameState, none_if_tie=True)
            nbs_ans = pm.run_nash_bargain(gameState, none_if_tie=True)

            if mec_ans not in results:
                results[mec_ans] = 0
            if nbs_ans not in results:
                results[nbs_ans] = 0

            results[mec_ans] += 1

            if mec_ans != nbs_ans:
                example_disagrees.append(gameState)
                disagreements += 1
                results[nbs_ans] += 1

            count += 1

    print(f'proportion disagree: {disagreements / count}, count: {disagreements}, n: {count}')
    print()
    print(results)
    return example_disagrees

In [33]:
ex_1 = test_disagreements(list(itertools.combinations(categories, 4)), sample_matchups,
                   mm_countries_sig, countries_pop_df)
print()
# sweet spot is here
test_disagreements(list(itertools.combinations(categories, 4)), sample_matchups,
                   mm_countries_sig, countries_count)
print()
ex_2 = test_disagreements(list(itertools.combinations(categories, 5)), sample_matchups,
                   mm_countries_sig, countries_pop_df)

proportion disagree: 0.004920634920634921, count: 62, n: 12600

{'Law [Illegal -> Legal]': 764, 'No. Characters [Less -> More]': 3520, 'Species [Pets -> Humans]': 5093, 'Fitness [Large -> Fit]': 81, 'Social Status [Low -> High]': 682, 'Age [Elderly -> Young]': 2503, 'Gender [Male -> Female]': 5, '[Passengers -> Pedestrians]': 14}

proportion disagree: 0.004285714285714286, count: 54, n: 12600

{'Law [Illegal -> Legal]': 810, 'Age [Elderly -> Young]': 2684, 'No. Characters [Less -> More]': 3297, 'Species [Pets -> Humans]': 5138, 'Fitness [Large -> Fit]': 85, 'Social Status [Low -> High]': 625, 'Gender [Male -> Female]': 9, '[Passengers -> Pedestrians]': 6}

proportion disagree: 0.007063492063492063, count: 89, n: 12600

{'Law [Illegal -> Legal]': 356, 'No. Characters [Less -> More]': 3600, 'Species [Pets -> Humans]': 6205, 'Social Status [Low -> High]': 288, 'Age [Elderly -> Young]': 2240}


### Drilling down on an example disagreement

In [25]:
pm.run_mec(ex_1[0])

'Species [Pets -> Humans]'

In [18]:
test_disagreements(list(itertools.combinations(categories, 4)), sample_matchups,
                   mm_countries_sig, countries_pop_df)

proportion disagree: 0.0037301587301587303

{'Law [Illegal -> Legal]': 787, 'Age [Elderly -> Young]': 2454, 'No. Characters [Less -> More]': 3478, 'Species [Pets -> Humans]': 5152, 'Gender [Male -> Female]': 9, 'Social Status [Low -> High]': 676, 'Fitness [Large -> Fit]': 79, '[Passengers -> Pedestrians]': 12}


In [12]:
# looking at the various rdata files

# tried this after download from https://osf.io/3hvt2/download
# but it is not interesting...
directory = '../external_data/moral_machines/Moral Machine Effect Sizes'
files = os.listdir(directory)
files
import pyreadr
result = pyreadr.read_r(os.path.join(directory, files[6]))
result

OrderedDict([('plotdata.util',
                 Estimates        se Variant           Label
              0   0.339062  0.001641       1  No. Characters
              1   0.489515  0.001322       2  No. Characters
              2   0.558371  0.000942       3  No. Characters
              3   0.644861  0.000842       4  No. Characters)])

one case or scenario for each two row (users are shown many cases)
each outcome is one of the two rows.... so dumb!
ResponseID: "a unique, random set of characters that represents an identifier of the scenario.
Since each scenario is represented by 2 rows,"

We probably just want to look at, say, 5 people vs. four people to begin with
but we need to extract the preferences for each country, say, for each attribute

> " - Saved: this resembles the actual decision made by the user [1: user decided to save the characters in this outcome, 0: user decided to kill the characters in this outcome]. Note that this column is reverse coded from the database. On the website, users click on the outcome they choose. That means the choice they make results in the death of the characters represented in that outcome (with a skull sign on the website). You can imagine another column named "Killed" which would be the exact opposite of "Saved" (i.e. 1 if Saved is 0 and 0 if Saved is 1)."

> - UserCountry3: the alpha-3 ISO code of the country from which the user accessed the website. This is generated from the user IP which is collected but not shared here.

> - ScenarioType and ScenarioTypeStrict: These two columns have 7 values, corresponding to 7 types of scenarios (6 attributes + random). These are: "Utilitarian","Gender", "Fitness", "Age", "Social Value", "Species", and "Random".
In the early stage of the website, we forgot to include a code that gives the scenario type (one of the 6 categories mentioned above + random). We had to write a code to figure that out from the character types. This is the "ScenarioType" column. Some scenarios who were generated as part of the "random", could fit in one of the 6 other categories. Later, we used a clear parameter to capture this type, which is in "ScenarioTypeStrict". Thus, this column provides an accurate description, but it does not have a value for the early scenarios. In the analysis for the figures, whenever we filtered based on the scenario type, we used both columns. For example, to filter the age related scenarios, we use: ScenarioTypeStrict=“Age” && ScenarioType=“Age” 
where "&&” is the logic AND.

Should read their nature paper again... figure out how to learn models for the preferences of each attribute

I think filter by scenario type then group by country and want count for each way... how did they determine the degree of preference for fitness at the end of the questionnaire?

Reading their supplementary information, need to compute the "average marginal component effect (AMCE)" this very complicated formula. Look at weakened formula on page 19

But I'm not sure that would be the appropriate weighting... 

They raise a good point about whether one demographic category is actually coding for the response of another demographic category and how that needs to be controlled for.

Consider the coder to generate 'Figure S8:' 

Easiest thing would be to simply copy 'Extended data table 1'

... make slides on this.

In [ ]:
df = pd.read_csv('../data/moral_machines/moral_machines_test.csv')

# need to do this as an interactive session
# dowload this repository, then download the data using the make file

# for each pair of rows
# verify that userid scenario id and session id are the same

# need 